In [1]:
import random

import ujson
from schemas.event import Event, Events, EventType
from shared.containers import init_combat_container
from shared.ulid import ulid_as_uuid

ModuleNotFoundError: No module named 'ujson'

In [2]:
container = init_combat_container()

/Users/teadove/projects/moscow_travel_hack_ecom_240323/backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/teadove/projects/moscow_travel_hack_ecom_240323/backend/.venv/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:293: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


In [3]:
cite_data = ujson.loads(open("cite_data.json").read())
restaurants = cite_data["restaurants"]
events = cite_data["events"]
reviews = ujson.loads(open("ml/data/with_reviews.json").read())
len(restaurants), len(events)

(9, 72)

In [4]:
name_to_reviews = reviews["restaurants"] + reviews["events"]
name_to_reviews = {
    reviews["name"]: reviews for reviews in reviews["restaurants"] + reviews["events"]
}
len(name_to_reviews)

45

In [5]:
events_parsed = []
for event in events:
    event["id"] = ulid_as_uuid()
    event["type"] = EventType.EVENT
    event["lat"] = event.get("latitude")
    event["lng"] = event.get("longitude")
    event["img_link"] = random.choice(
        [
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F65420ad0d2b6381af066b9f9%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F65420b82d2b6381af066dba8%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F652400ac2faec1be5888a10a%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F623982d6994c9997315b67ab%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F623982c6994c9997315b66ff%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F623982da98e199bbd30c92c1%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F660fcfe8e6f042f1b71d4dd3%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F660a8004e8caad7a6aba7062%2F924&w=3840&q=75",
            "https://russpass.ru/mesta-i-sobytiya/_next/image?url=https%3A%2F%2Fcms.russpass.ru%2Fv1%2Ffile%2F65b39ea810de61faf900dd4f%2F924&w=3840&q=75",
        ]
    )
    review = name_to_reviews.get(event["name"])
    if review is not None:
        event["reviews"] = review["reviews"]

    events_parsed.append(Event.parse_obj(event))

len(events_parsed)

72

In [6]:
restaurants_parsed = []
for restaurant in restaurants:
    restaurant["id"] = ulid_as_uuid()
    restaurant["restaurant_type"] = restaurant.get("type")
    restaurant["type"] = EventType.RESTAURANT
    restaurant["lat"] = restaurant.get("latitude")
    restaurant["lng"] = restaurant.get("longitude")
    restaurant["img_link"] = random.choice(
        [
            "https://cms.russpass.ru/v1/file/657aed9a3796b43717403233",
            "https://cms.russpass.ru/v1/file/657aed9a3796b43717403213",
            "https://cms.russpass.ru/v1/file/658159e398923097942d3731",
        ]
    )
    restaurants_parsed.append(Event.parse_obj(restaurant))

len(restaurants_parsed)
[restaurants.restaurant_type for restaurants in restaurants_parsed]

[['Испанская'],
 ['Русская'],
 ['Итальянская', 'Европейская', 'Авторская'],
 ['Русская'],
 ['Авторская', 'Кавказская'],
 ['Грузинская'],
 ['Русская'],
 ['Европейская', 'Японская', 'Паназиатская', 'Китайская'],
 ['Грузинская']]

In [34]:
events = Events(events=restaurants_parsed + events_parsed)

In [7]:
container.heath_service.pg_repository.insert_events(events)

AttributeError: 'list' object has no attribute 'events'

In [36]:
len(container.heath_service.pg_repository.get_events().events)

162